<a href="https://colab.research.google.com/github/pamela2412/-Cria-o-de-base-de-dados-e-treinamento-da-rede-YOLO/blob/main/Cria%C3%A7%C3%A3o_de_base_de_dados_e_treinamento_da_rede_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.1. Instalar YOLOv5 e Dependências

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.11 MiB | 21.28 MiB/s, done.
Resolving deltas: 100% (11861/11861), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 664.8/664.8 MB 158.4 MB/s eta 0:00:01

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Imagens de treino e validação
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip

# Anotações
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

# Descompactar
!unzip train2017.zip -d /content/dataset
!unzip val2017.zip -d /content/dataset
!unzip annotations_trainval2017.zip -d /content/dataset

In [ ]:
import os
import json
from pycocotools.coco import COCO

# Diretórios de entrada/saída
annotations_path = "/content/dataset/annotations/instances_train2017.json"
output_dir = "/content/dataset/train/labels"

# Classes desejadas (nomes exatos do COCO)
desired_classes = ["bird", "cat", "dog"]  # Usar os nomes originais do COCO!

# Carregar anotações COCO
coco = COCO(annotations_path)

# Obter IDs das categorias desejadas
categories = coco.loadCats(coco.getCatIds())
category_name_to_id = {cat['name']: cat['id'] for cat in categories}

# Filtrar IDs das classes desejadas
desired_class_ids = [category_name_to_id[cls] for cls in desired_classes]

# Mapear IDs originais do COCO para novos IDs do YOLO (0, 1, 2)
original_to_yolo_id = {
    orig_id: new_id for new_id, orig_id in enumerate(desired_class_ids)
}

# Criar pasta de saída
os.makedirs(output_dir, exist_ok=True)

# Processar cada imagem
for img_id in coco.getImgIds():
    img_info = coco.loadImgs(img_id)[0]
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=desired_class_ids)  # Filtra por classe
    annotations = coco.loadAnns(ann_ids)

    # Caminho do arquivo .txt
    txt_path = os.path.join(output_dir, img_info['file_name'].replace('.jpg', '.txt'))

    # Escrever apenas se houver anotações
    if len(annotations) > 0:
        with open(txt_path, 'w') as f:
            for ann in annotations:
                # Converter bbox COCO para YOLO
                x, y, w, h = ann['bbox']
                img_width = img_info['width']
                img_height = img_info['height']

                x_center = (x + w/2) / img_width
                y_center = (y + h/2) / img_height
                w_norm = w / img_width
                h_norm = h / img_height

                # Mapear ID da classe para YOLO
                yolo_class_id = original_to_yolo_id[ann['category_id']]

                f.write(f"{yolo_class_id} {x_center} {y_center} {w_norm} {h_norm}\n")

In [ ]:
import os

# Crear carpetas necesarias
os.makedirs("/content/dataset/train/images", exist_ok=True)
os.makedirs("/content/dataset/train/labels", exist_ok=True)
os.makedirs("/content/dataset/val/images", exist_ok=True)
os.makedirs("/content/dataset/val/labels", exist_ok=True)

In [ ]:
import os
import shutil

# Rutas
src_train = "/content/dataset/train2017"
dst_train = "/content/dataset/train/images"

src_val = "/content/dataset/val2017"
dst_val = "/content/dataset/val/images"

# Mover archivos .jpg de train2017
for filename in os.listdir(src_train):
    if filename.endswith(".jpg"):
        shutil.move(os.path.join(src_train, filename), os.path.join(dst_train, filename))

# Mover archivos .jpg de val2017
for filename in os.listdir(src_val):
    if filename.endswith(".jpg"):
        shutil.move(os.path.join(src_val, filename), os.path.join(dst_val, filename))

In [ ]:
%%writefile /content/dataset/data.yaml
train: /content/dataset/train/images
val: /content/dataset/val/images
nc: 3
names: ['bird', 'cat', 'dog']

In [ ]:
!python yolov5/train.py \
  --img 640 \
  --batch 16 \
  --epochs 10 \
  --data /content/dataset/data.yaml \
  --cfg yolov5/models/yolov5s.yaml \
  --weights yolov5s.pt \
  --name treino_bird_cat_dog

In [ ]:
!python yolov5/detect.py \
  --weights yolov5/runs/train/treino_bird_cat_dog/weights/best.pt \  # Ruta de los pesos entrenados
  --source /content/imagen_2025-02-16_190005481.png \           # Imagen de prueba
  --conf 0.5       # Umbral de confianz
  --img 640 \
  --save-txt \
  --save-conf

IndentationError: unexpected indent (<ipython-input-2-d72b8603cdbc>, line 2)